In [1]:
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers, metrics

from model_train import load_c3d_model, train_msupcl_model, linear_evaluation
from data_uniform_sup import VideoDataGenerator
from paired_generator import PairedDataGenerator

In [2]:
seed = 2042
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
input_shape = (16, 112, 112, 3)  # As defined in data generator
num_classes = 2  # Harmful or Safe

In [3]:
# Define dataset paths
violence_negative_dir = './data/violence_dataset/NonViolence'
violence_positive_dir = './data/violence_dataset/Violence'
tiktok_negative_dir = './data/tiktok/train/Safe'
tiktok_positive_dir = './data/tiktok/train/Harmful Content'


In [4]:
def sample_videos(directory, num_samples=100):
    all_videos = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mp4')]
    sampled_videos = random.sample(all_videos, min(num_samples, len(all_videos)))
    return sampled_videos


In [5]:
# Violence dataset
violence_negative_videos = sample_videos(violence_negative_dir, 100)
violence_positive_videos = sample_videos(violence_positive_dir, 100)

# TikTok dataset
tiktok_negative_videos = sample_videos(tiktok_negative_dir, 100)
tiktok_positive_videos = sample_videos(tiktok_positive_dir, 100)


In [6]:
def split_data(negative_videos, positive_videos, train_ratio=0.55, val_ratio=0.15):
    # Combine and shuffle
    videos = negative_videos + positive_videos
    labels = [0]*len(negative_videos) + [1]*len(positive_videos)
    combined = list(zip(videos, labels))
    random.shuffle(combined)
    videos[:], labels[:] = zip(*combined)
    
    # Calculate split indices
    total = len(videos)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    
    # Split data
    train_videos = videos[:train_end]
    train_labels = labels[:train_end]
    val_videos = videos[train_end:val_end]
    val_labels = labels[train_end:val_end]
    test_videos = videos[val_end:]
    test_labels = labels[val_end:]
    
    return (train_videos, train_labels), (val_videos, val_labels), (test_videos, test_labels)


In [7]:
# Violence dataset
(violence_train_videos, violence_train_labels), \
(violence_val_videos, violence_val_labels), \
(violence_test_videos, violence_test_labels) = split_data(violence_negative_videos, violence_positive_videos)

# TikTok dataset
(tiktok_train_videos, tiktok_train_labels), \
(tiktok_val_videos, tiktok_val_labels), \
(tiktok_test_videos, tiktok_test_labels) = split_data(tiktok_negative_videos, tiktok_positive_videos)


In [8]:

# Convert labels to numpy arrays and one-hot encode them if necessary
def prepare_labels(labels):
    return np.array(labels)

# Violence dataset generators
violence_train_labels_np = prepare_labels(violence_train_labels)
violence_val_labels_np = prepare_labels(violence_val_labels)
violence_test_labels_np = prepare_labels(violence_test_labels)

violence_train_generator = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=4, shuffle=True, augment=True)
violence_val_generator = VideoDataGenerator(violence_val_videos, violence_val_labels_np, batch_size=4, shuffle=False)
violence_test_generator = VideoDataGenerator(violence_test_videos, violence_test_labels_np, batch_size=4, shuffle=False)

In [9]:

# TikTok dataset generators
tiktok_train_labels_np = prepare_labels(tiktok_train_labels)
tiktok_val_labels_np = prepare_labels(tiktok_val_labels)
tiktok_test_labels_np = prepare_labels(tiktok_test_labels)

tiktok_train_generator = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=4, shuffle=True, augment=True)
tiktok_val_generator = VideoDataGenerator(tiktok_val_videos, tiktok_val_labels_np, batch_size=4, shuffle=False)
tiktok_test_generator = VideoDataGenerator(tiktok_test_videos, tiktok_test_labels_np, batch_size=4, shuffle=False)


In [10]:

# Load the model
model = load_c3d_model(input_shape=input_shape, feature_dim=512)


In [11]:
# Modify the model to output class probabilities


# Freeze the base model if desired
for layer in model.layers:
    layer.trainable = False

# Add classification layer
features = model.output
outputs = layers.Dense(num_classes, activation='softmax')(features)
classification_model = models.Model(inputs=model.input, outputs=outputs)

# Compile the model
classification_model.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=[metrics.SparseCategoricalAccuracy()]
)


In [12]:
# Train on Violence dataset
history_violence = classification_model.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=10
)


Epoch 1/10
27/27 [==============================] - 24s 728ms/step - loss: 0.6936 - sparse_categorical_accuracy: 0.4630 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.4643
Epoch 2/10
27/27 [==============================] - 20s 734ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4815 - val_loss: 0.6935 - val_sparse_categorical_accuracy: 0.4643
Epoch 3/10
27/27 [==============================] - 20s 724ms/step - loss: 0.6935 - sparse_categorical_accuracy: 0.5046 - val_loss: 0.6936 - val_sparse_categorical_accuracy: 0.4643
Epoch 4/10
27/27 [==============================] - 20s 744ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4491 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.4643
Epoch 5/10
27/27 [==============================] - 20s 744ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4954 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.4643
Epoch 6/10
27/27 [==============================] - 20s 738ms/step - loss: 0.6936 - sparse_cate

In [13]:
# Evaluate on Violence test set
results_violence = classification_model.evaluate(violence_test_generator)
print(f"Violence Dataset - Test Loss: {results_violence[0]}, Test Accuracy: {results_violence[1]}")


15/15 [==============================] - 9s 584ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5167
Violence Dataset - Test Loss: 0.6931942701339722, Test Accuracy: 0.5166666507720947


In [14]:
history_tiktok = classification_model.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=10
)

# Evaluate on TikTok test set
results_tiktok = classification_model.evaluate(tiktok_test_generator)
print(f"TikTok Dataset - Test Loss: {results_tiktok[0]}, Test Accuracy: {results_tiktok[1]}")

Epoch 1/10
27/27 [==============================] - 61s 2s/step - loss: 0.6945 - sparse_categorical_accuracy: 0.4120 - val_loss: 0.6929 - val_sparse_categorical_accuracy: 0.6071
Epoch 2/10
27/27 [==============================] - 59s 2s/step - loss: 0.6930 - sparse_categorical_accuracy: 0.4769 - val_loss: 0.6932 - val_sparse_categorical_accuracy: 0.4643
Epoch 3/10
27/27 [==============================] - 60s 2s/step - loss: 0.6934 - sparse_categorical_accuracy: 0.4769 - val_loss: 0.6934 - val_sparse_categorical_accuracy: 0.2857
Epoch 4/10
27/27 [==============================] - 60s 2s/step - loss: 0.6925 - sparse_categorical_accuracy: 0.5046 - val_loss: 0.6937 - val_sparse_categorical_accuracy: 0.3929
Epoch 5/10
27/27 [==============================] - 61s 2s/step - loss: 0.6918 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.3929
Epoch 6/10
27/27 [==============================] - 60s 2s/step - loss: 0.6918 - sparse_categorical_accuracy: 

## MSupCL implementation

In [14]:
# Combine training data from both datasets
combined_train_videos = violence_train_videos + tiktok_train_videos
combined_train_labels = violence_train_labels_np.tolist() + tiktok_train_labels_np.tolist()

# Create a combined data generator
combined_train_generator = VideoDataGenerator(combined_train_videos, combined_train_labels, batch_size=4, shuffle=True, augment=True)


In [11]:


# Create separate generators for violence and tiktok datasets
violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=4, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=4, shuffle=True, augment=False)

# Create paired data generator
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)




In [12]:
# Load the model
msupcl_model = load_c3d_model(input_shape=input_shape)

# Train the model
train_msupcl_model(msupcl_model, paired_train_generator, epochs=10)


Epoch 1/10
Training Loss: 0.7348
Epoch 2/10
Training Loss: 0.7323
Epoch 3/10
Training Loss: 0.7217
Epoch 4/10
Training Loss: 0.7297
Epoch 5/10
Training Loss: 0.7315
Epoch 6/10
Training Loss: 0.7242
Epoch 7/10
Training Loss: 0.7307
Epoch 8/10
Training Loss: 0.7272
Epoch 9/10
Training Loss: 0.7261
Epoch 10/10
Training Loss: 0.7242


In [13]:
linear_evaluation(msupcl_model, combined_train_generator,violence_val_generator, tiktok_val_generator)


Epoch 1/5
55/55 [==============================] - 72s 1s/step - loss: 11.0498 - sparse_categorical_accuracy: 0.4932 - val_loss: 0.7518 - val_sparse_categorical_accuracy: 0.5357
Epoch 2/5
55/55 [==============================] - 72s 1s/step - loss: 6.7122 - sparse_categorical_accuracy: 0.4886 - val_loss: 1.8705 - val_sparse_categorical_accuracy: 0.5357
Epoch 3/5
55/55 [==============================] - 70s 1s/step - loss: 7.0818 - sparse_categorical_accuracy: 0.4841 - val_loss: 1.6976 - val_sparse_categorical_accuracy: 0.5357
Epoch 4/5
55/55 [==============================] - 73s 1s/step - loss: 6.0086 - sparse_categorical_accuracy: 0.5045 - val_loss: 0.6971 - val_sparse_categorical_accuracy: 0.4286
Epoch 5/5
55/55 [==============================] - 73s 1s/step - loss: 5.8902 - sparse_categorical_accuracy: 0.5159 - val_loss: 0.7459 - val_sparse_categorical_accuracy: 0.5000
Evaluating on Violence Test Set:
7/7 [==============================] - 5s 643ms/step - loss: 0.7155 - sparse_cate

## R2+1d_18 model

In [10]:
from model_train_r2plus1d_18 import load_r2plus1d_model, linear_evaluation, train_msupcl_model, linear_evaluation

input_shape = (16, 112, 112, 3)
feature_dim = 512

msupcl_model = load_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim, include_top=False)

In [11]:
batch_size = 4

violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)

In [12]:
train_msupcl_model(msupcl_model, paired_train_generator, epochs=10)

Epoch 1/10
Training Loss: 0.7068
Epoch 2/10
Training Loss: 0.7050
Epoch 3/10
Training Loss: 0.7007
Epoch 4/10
Training Loss: 0.6996
Epoch 5/10
Training Loss: 0.6959
Epoch 6/10
Training Loss: 0.6910
Epoch 7/10
Training Loss: 0.7074
Epoch 8/10
Training Loss: 0.6976
Epoch 9/10
Training Loss: 0.7011
Epoch 10/10
Training Loss: 0.6987


In [15]:
linear_evaluation(msupcl_model, combined_train_generator, violence_val_generator,tiktok_val_generator, num_classes=2)

Epoch 1/5
55/55 [==============================] - 71s 1s/step - loss: 1.0014 - sparse_categorical_accuracy: 0.4727 - val_loss: 0.7135 - val_sparse_categorical_accuracy: 0.4643
Epoch 2/5
55/55 [==============================] - 69s 1s/step - loss: 0.6855 - sparse_categorical_accuracy: 0.5432 - val_loss: 0.7238 - val_sparse_categorical_accuracy: 0.4286
Epoch 3/5
55/55 [==============================] - 69s 1s/step - loss: 0.6928 - sparse_categorical_accuracy: 0.5386 - val_loss: 0.7181 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/5
55/55 [==============================] - 69s 1s/step - loss: 0.6909 - sparse_categorical_accuracy: 0.5227 - val_loss: 0.7405 - val_sparse_categorical_accuracy: 0.3571
Epoch 5/5
55/55 [==============================] - 72s 1s/step - loss: 0.6916 - sparse_categorical_accuracy: 0.5341 - val_loss: 0.7396 - val_sparse_categorical_accuracy: 0.5000
Evaluating on Violence Test Set:
7/7 [==============================] - 4s 624ms/step - loss: 0.7471 - sparse_categ